In [1]:
import numpy as np

#Coviddetector
from coviddet.generator.dataset import Dataset
from coviddet.generator.dataloader import Dataloder
from coviddet.models.architecture import CovidClassifier

# Rest
import pandas as pd
from typing import List

# model definition
import PIL
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import ResNet50, DenseNet121, Xception
from keras.layers import Input, Conv2D

# Preprocessing and additionals:
from sklearn.preprocessing import MinMaxScaler
from skimage import exposure
scaler = MinMaxScaler()

import cv2

import random

num_classes = 4

# matplotlib: 
import matplotlib.pyplot as plt

# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
# Set GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"

model_archive = '../trained_models/'

In [2]:
def decode_label(labels, classes):
    if labels.sum()==0:
        return 'Not Classified'
    for label, name in zip(labels, classes):
        if label==1:
            return str(name)

In [3]:
dataset_path = '../dataset.json'
dataset_images = '../../../covid_dataset/'

In [4]:
train, validation, test = Dataset.create_datasets(dataset_images, dataset_path, 0.1, 200)

dataset length: 6054
Evaluation Cases: 200
Train cases: 5269
Validation cases: 585


In [5]:
train_gen = Dataloder(train, batch_size=16, preprocessing='MinMax', shuffle=True, expanddims=True)
valid_gen = Dataloder(train, batch_size=8, preprocessing='MinMax', shuffle=False, expanddims=True)

In [6]:
LR = 0.0001
optim = keras.optimizers.Adam(LR)

In [7]:
classifier = CovidClassifier()

2022-01-20 21:34:39.166380: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-20 21:34:39.633534: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38414 MB memory:  -> device: 0, name: A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0


512.0
256.0
128.0
64.0
32.0
16.0
8.0
4.0


In [8]:
metrics = [keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None), 
                        keras.metrics.Precision(name="precision"), 
                        keras.metrics.Recall(name="recall")]

classifier.compile(optimizer=optim,
                     loss='hinge',# keras.losses.BinaryCrossentropy(), # default from_logits=False
                     metrics=metrics)

classifier.build(train_gen[0][0].shape)

In [18]:
# Callbacks:
checkpoint_dir = os.path.join(model_archive, classifier.name)

if os.path.exists(checkpoint_filepath):
    print("All weights will be stored under {}".format(checkpoint_dir))
    
else:
    os.mkdir(os.path.join(model_archive, classifier.name))
    print("Directory for model {} created!".format(classifier.name))
    
checkpoint_file = os.path.join(checkpoint_dir, 'model_weights')

checkpoint_save = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_file,
    monitor="categorical_accuracy",
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode="auto",
    save_freq="epoch",
    options=None,
)

All weights will be stored under ../trained_models/covid_classifier


In [19]:
classifier.summary()

Model: "covid_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Converter (Conv2D)          multiple                  6         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 GLobAvaPooling2D (GlobalAve  multiple                 0         
 ragePooling2D)                                                  
                                                                 
 Sequential_Block (Sequentia  (16, 8)                  2798200   
 l)                                                              
                                                                 
 ModelOutput (Dense)         multiple                  36        
                                                                 
Total params: 26,385,954
Trainable params: 26,332,

In [21]:
history = classifier.fit(train_gen,
                        validation_data=valid_gen,
                        epochs=10,
                        callbacks=[checkpoint_save])

Epoch 1/10


2022-01-20 21:46:14.531038: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-01-20 21:46:16.643306: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


 38/329 [==>...........................] - ETA: 14:36 - loss: 1.0571 - categorical_accuracy: 0.4655 - precision: 0.4913 - recall: 0.3240

/home/mstryja/.conda/envs/stryja_sat/lib/python3.7/site-packages/pydicom/pixel_data_handlers/numpy_handler.py:359: UserWarning: The length of the pixel data in the dataset (13262360 bytes) indicates it contains excess padding. 216296 bytes will be removed from the end of the data
  warnings.warn(msg)


329/329 [==============================] - ETA: 0s - loss: 1.0205 - categorical_accuracy: 0.4683 - precision: 0.4704 - recall: 0.4519

...

329/329 [==============================] - 1890s 6s/step - loss: 1.0151 - categorical_accuracy: 0.4698 - precision: 0.4698 - recall: 0.4698 - val_loss: 1.0151 - val_categorical_accuracy: 0.4698 - val_precision: 0.4698 - val_recall: 0.4698
Epoch 10/10
259/329 [======================>.......] - ETA: 3:26 - loss: 1.0142 - categorical_accuracy: 0.4715 - precision: 0.4715 - recall: 0.4715

/home/mstryja/.conda/envs/stryja_sat/lib/python3.7/site-packages/pydicom/pixel_data_handlers/numpy_handler.py:359: UserWarning: The length of the pixel data in the dataset (13262360 bytes) indicates it contains excess padding. 216296 bytes will be removed from the end of the data
  warnings.warn(msg)


329/329 [==============================] - ETA: 0s - loss: 1.0151 - categorical_accuracy: 0.4698 - precision: 0.4698 - recall: 0.4698

/home/mstryja/.conda/envs/stryja_sat/lib/python3.7/site-packages/pydicom/pixel_data_handlers/numpy_handler.py:359: UserWarning: The length of the pixel data in the dataset (13262360 bytes) indicates it contains excess padding. 216296 bytes will be removed from the end of the data
  warnings.warn(msg)


329/329 [==============================] - 1911s 6s/step - loss: 1.0151 - categorical_accuracy: 0.4698 - precision: 0.4698 - recall: 0.4698 - val_loss: 1.0151 - val_categorical_accuracy: 0.4698 - val_precision: 0.4698 - val_recall: 0.4698


In [ ]:
# classifier.load_weights(os.path.join(model_archive, 'covid_classifier_2', 'model_final_weights.h5'), by_name=True)

In [ ]:
# Tomorrow balance dataset using image augmentation - on the Dataset stage!